## 一个简单的Seq2Seq例子：将中文翻译成英文

In [1]:
from keras.models import Model
from keras.layers import Input, CuDNNLSTM, Dense
from keras import callbacks
import numpy as np

Using TensorFlow backend.


In [2]:
# 基本参数
batch_size=64
epochs= 100
latent_dim= 256 # LSTM单元数量
num_samples= 10000#训练样本大小
dataset="cmn.txt"

In [3]:
#数据读取
input_texts=[]
target_texts= []
input_characters= set()
target_characcters= set()
with open(dataset,"r",encoding="utf-8") as f:
    lines= f.read().split("\n")

In [4]:
lines[:20]

['Hi.\t嗨。',
 'Hi.\t你好。',
 'Run.\t你用跑的。',
 'Wait!\t等等！',
 'Hello!\t你好。',
 'I try.\t让我来。',
 'I won!\t我赢了。',
 'Oh no!\t不会吧。',
 'Cheers!\t乾杯!',
 'Got it?\t你懂了吗？',
 'He ran.\t他跑了。',
 'Hop in.\t跳进来。',
 'I lost.\t我迷失了。',
 'I quit.\t我退出。',
 "I'm OK.\t我沒事。",
 'Listen.\t听着。',
 'No way!\t不可能！',
 'No way!\t没门！',
 'Really?\t你确定？',
 'Try it.\t试试吧。']

In [5]:
# 分割lines，得到训练数据
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text, target_text=line.split("\t")
    target_text= "\t"+target_text+"\n"#用"\t"作为序列开始标志，"\n"作为序列结束标志
    input_texts.append(input_text)
    target_texts.append(target_text)
    #计算input_text中的tokens,英文中的tokens是字符级别
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    #计算target_text中的tokens
    for char in target_text:
        if char not in target_characcters:
            target_characcters.add(char)

In [6]:
input_characters= sorted(list(input_characters))
target_characcters= sorted(list(target_characcters))
num_encoder_tokens= len(input_characters)
num_decoder_tokens= len(target_characcters)
max_encoder_seq_length= max([len(txt) for txt in input_texts])
max_decoder_seq_length= max([len(txt) for txt in target_texts])

In [7]:
target_characcters[60:70]

['三', '上', '下', '不', '与', '丐', '丑', '专', '且', '世']

你说中文是怎么进行sorted的？依据ascii码？这就得了解汉字在计算机中的编码方式

In [8]:
print("样本数量：",len(input_texts))
print("输入tokens数量：",num_encoder_tokens)
print("目标tokens数量：",num_decoder_tokens)
print("输入最大长度：",max_encoder_seq_length)
print("输出最大长度：",max_decoder_seq_length)

样本数量： 10000
输入tokens数量： 73
目标tokens数量： 2580
输入最大长度： 30
输出最大长度： 22


In [9]:
#建立字符-数字字典用于字符向量化
input_token_index= dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index= dict([(char,i) for i,char in enumerate(target_characcters)])
#创建数组
encoder_input_data= np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype=np.float32)
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype=np.float32)
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype=np.float32)
#填充数据，对每一个字符做one-hot
for i, (input_text, target_text) in enumerate(zip(input_texts,target_texts)):
    #对编码器序列做One-hot
    for t, char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]]=1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]]=1.0
        if t>0:
            decoder_target_data[i,t-1,target_token_index[char]]=1.0

In [10]:
#定义编码器输入
encoder_inputs= Input(shape=(None, num_encoder_tokens))
#编码器
encoder= CuDNNLSTM(latent_dim,return_state=True)
#调用编码器，得到编码器输入、状态信息
encoder_output, state_h, state_c= encoder(encoder_inputs)
#丢弃编码器的输出，我们只需要编码器的状态
encoder_state= [state_h,state_c]

#定义解码器输入
decoder_inputs= Input(shape=(None, num_decoder_tokens))
decoder_lstm= CuDNNLSTM(latent_dim, return_sequences=True, return_state= True)
#将编码器输出的状态作为解码器的初始状态
decoder_outputs, _,_= decoder_lstm(decoder_inputs,initial_state=encoder_state)
#添加全连接层
decoder_dense= Dense(num_decoder_tokens, activation="softmax")
decoder_outputs= decoder_dense(decoder_outputs)

#定义整个模型
model= Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.compile(optimizer="rmsprop",loss="categorical_crossentropy")

In [ ]:
#训练
history=model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,epochs= epochs,validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 22s 3ms/step - loss: 1.9675 - val_loss: 2.5045
Epoch 2/100
8000/8000 [==============================] - 12s 2ms/step - loss: 1.8368 - val_loss: 2.4496
Epoch 3/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.7354 - val_loss: 2.2843
Epoch 4/100
8000/8000 [==============================] - 12s 1ms/step - loss: 1.6451 - val_loss: 2.2005
Epoch 5/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.5577 - val_loss: 2.1139
Epoch 6/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.4787 - val_loss: 2.0716
Epoch 7/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.4121 - val_loss: 2.0083
Epoch 8/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.3489 - val_loss: 1.9546
Epoch 9/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.2926 - val_loss: 1.9197
Epoch 10/100
800

In [ ]:
model.save("seq2seq.h5")